# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))

# check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster and establishing a connection

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS Project2
                WITH REPLICATION= 
                {'class':'SimpleStrategy', 'replication_factor':1}""")

#### Set Keyspace

In [9]:
session.set_keyspace('project2')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [10]:
query ="""CREATE TABLE IF NOT EXISTS song_info_by_session """
query = query+ """(sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))"""

try:
    session.execute(query)
except Exception as e:
    print(e)
    
select_query="select artist, song_title, song_length from song_info_by_session where sessionId=%s and itemInSession=%s"

                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_info_by_session (sessionId , itemInSession , artist , song_title ,song_length )"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
try:
    rows = session.execute(select_query, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song_title, "\nSong Length: "+str(row.song_length))

Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


## Question 2:


### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    


In [14]:
query_1 ="""CREATE TABLE IF NOT EXISTS artist_info_by_user_session """
query_1 = query_1+ """(userId int, sessionId int, itemInSession int, artist_name text, firstName text, lastName text,  song_title text, PRIMARY KEY((userId, sessionId), itemInSession))"""

try:
    session.execute(query_1)
except Exception as e:
    print(e)


select_query_1= "select artist_name, song_title, firstName, lastName from artist_info_by_user_session where userId=%s and sessionId=%s"

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO artist_info_by_user_session (userId, sessionId, itemInSession,artist_name, firstName, lastName,  song_title)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]),int(line[8]), int(line[3]), line[0], line[1], line[4], line[9])) 


In [15]:
try:
    rows = session.execute(select_query_1, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist_name, "\nSong Title: "+row.song_title, "\nFirst Name: "+str(row.firstname), "\nLast Name: "+str(row.lastname))

Artist: Down To The Bone 
Song Title: Keep On Keepin' On 
First Name: Sylvie 
Last Name: Cruz
Artist: Three Drives 
Song Title: Greece 2000 
First Name: Sylvie 
Last Name: Cruz
Artist: Sebastien Tellier 
Song Title: Kilometer 
First Name: Sylvie 
Last Name: Cruz
Artist: Lonnie Gordon 
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
First Name: Sylvie 
Last Name: Cruz


## Question 3

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
query_2 ="""CREATE TABLE IF NOT EXISTS song_listens_per_user """
query_2 = query_2 + """(song_title text, userId int, firstName text, lastName text, PRIMARY KEY((song_title), userId))"""

try:
    session.execute(query_2)
except Exception as e:
    print(e)

select_query_2= """select firstName, lastName from song_listens_per_user where song_title = 'All Hands Against His Own'"""


In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip
    for line in csvreader:
        query = "INSERT INTO song_listens_per_user (song_title, userId,  firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9],int(line[10]), line[1], line[4] )) 

In [18]:
try:
    rows = session.execute(select_query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print("\nFirst Name: "+str(row.firstname), "\nLast Name: "+str(row.lastname))


First Name: Jacqueline 
Last Name: Lynch

First Name: Tegan 
Last Name: Levine

First Name: Sara 
Last Name: Johnson


### Drop the tables before closing out the sessions

In [19]:
session.execute("DROP TABLE IF EXISTS song_info_by_session")
session.execute("DROP TABLE IF EXISTS artist_info_by_user_session")
session.execute("DROP TABLE IF EXISTS song_listens_per_user")

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()